<a href="https://colab.research.google.com/github/ferustigris/stock/blob/master/Update_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

Install dependencies for Yahoo finance and gsheets. As well as client_secret.json required in root directory. Could be downloaded from [here](https://console.cloud.google.com/apis/api/drive.googleapis.com/metrics?project=gold-access-268714)

In [11]:
!pip install pygsheets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Framework

Create Yahoo API reader:

In [32]:
import yfinance as yf

BLNS = 1000_000_000

class NoValue(Exception):
  pass

class YahooFinanceReader:
    """Yahoo Finance Reader class for access via API"""

    class Ticker:
        """Case-class for the particular ticket"""

        def __init__(self, tkt):
            """Pass ticket object in the constructor"""
            self.tkt = tkt

        def get_current_price(self):
            """Get very current price of the ticket"""
            return self.tkt.info["currentPrice"]

        def get_divs(self):
            """Get annual divs of the ticket"""
            # print(self.tkt.dividends)
            divs = self.tkt.info["trailingAnnualDividendYield"]
            print("divs:")
            print(divs)
            if not divs:
              try:
                divs = self.tkt.info["dividendYield"]
                print("dividendYield:")
                print(divs)
              except KeyError:
                print("dividendYield: key error")
            if not divs:
              try:
                divs = self.tkt.info["forwardAnnualDividendYield"]
                print("forwardAnnualDividendYield:")
                print(divs)
              except KeyError:
                print("forwardAnnualDividendYield: key error")
            return divs

        def get_net(self):
            """Get annual netIncomeToCommon of the ticket in billions"""
            for i in range(3):
              try:
                return self.tkt.info["netIncomeToCommon"] / BLNS
              except KeyError:
                print("ERROR: %s" % "netin key error")
            raise NoValue("ERROR: %s" % "netin key error")

        def get_revenue(self):
            for i in range(3):
              try:
                """Get annual totalRevenue of the ticket in billions"""
                return self.tkt.info["totalRevenue"] / BLNS
              except KeyError:
                print("ERROR: %s" % "Total revenue key error")
            raise NoValue("ERROR: %s" % "Total revenue key error")

        def get_assets(self):
            """Get total assets of the ticket in billions"""
            return self.tkt.info["totalAssets"] / BLNS

        def get_shares(self):
            """Get shares volume of the ticket in billions"""
            return self.tkt.info["sharesOutstanding"] / BLNS

        def get_sma200days(self):
            """Get twoHundredDayAverage of the ticket in billions"""
            return self.tkt.info["twoHundredDayAverage"] / BLNS

        def get_history(self):
          return self.tkt.history(period="max")

    def get_ticker(self, ticker_name):
        """Returns ticket object by ticket name"""
        print("Getting ticket object for %s..." % ticker_name)
        return self.Ticker(yf.Ticker(ticker_name))


Create gsheet writer, the file [itself](https://docs.google.com/spreadsheets/d/1hc8kcZJDUJ91qL-mp_NDxZUkrOnhihdzNhxVKRF04fQ/edit#gid=1283370971):

In [28]:
import pygsheets

# authorization
MAX_AMOUNT_ROWS = 50
COL_PRICE = 'E'
COL_DIVS = 'V'
COL_REVENUE = 'L'
COL_NET = 'M'
COL_ASSETS = 'O'
COL_SHARES = 'D'
COL_AVG = 'F'
NYSE_SHEET_NUMBER = 1
EU_SHEET_NUMBER = 3
PATH_TO_KEY_JSON = 'client_secret.json'
GSHEET_NAME = 'Finance stock and invest'


class GSheetUpdater:
    """Class for Finance stock and invest updates"""
    def __init__(self, sheet=NYSE_SHEET_NUMBER):
        self.__gc = pygsheets.authorize(service_file=PATH_TO_KEY_JSON)
        # open the google spreadsheet (by is the name of my sheet)
        self.__sh = self.__gc.open(GSHEET_NAME)

        # select the right sheet
        self.__wks = self.__sh[sheet]

    def apply_for_all(self, func, col_to_be_updated):
        """applies function for all rows"""
        for i in range(2, MAX_AMOUNT_ROWS):
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            try:
              self.__wks.update_value('%s%i' % (col_to_be_updated, i), try_func(ticket))
            except NoValue:
              print("ERROR: %s" % s)

    def apply_for_all(self, cols_and_funcs, start_from=2):
        """applies function for all rows"""
        for i in range(start_from, MAX_AMOUNT_ROWS):
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            if not ticket:
              raise NoValue("No ticket found")
            for (col_to_be_updated, func) in cols_and_funcs.items():
                print("Index=%i" % i)
                try:
                  self.__wks.update_value('%s%i' % (col_to_be_updated, i), func(ticket))
                except NoValue:
                    print("ERROR: %s" % s)


# Update stocks

What we are going to update:

In [33]:
r = YahooFinanceReader()


def update_price(ticket):
    """Apply the function for every ticker"""
    print("Update price for %s" % ticket)
    price = r.get_ticker(ticket).get_current_price()
    return price


cols_and_funcs = {
    COL_PRICE: update_price,
    COL_DIVS: lambda ticket: r.get_ticker(ticket).get_divs(),
    COL_REVENUE: lambda ticket: r.get_ticker(ticket).get_revenue(),
    # COL_ASSETS: lambda ticket: r.get_ticker(ticket).get_assets(),
    COL_SHARES: lambda ticket: r.get_ticker(ticket).get_shares(),
    COL_NET: lambda ticket: r.get_ticker(ticket).get_net(),
    COL_AVG: lambda ticket: r.get_ticker(ticket).get_sma200days()
}


Updating gsheet for EU:

In [ ]:
sheet_eu = GSheetUpdater(EU_SHEET_NUMBER)
sheet_eu.apply_for_all(cols_and_funcs, 2)

Updating gsheet for NYSE:

In [35]:
sheet_ny = GSheetUpdater(NYSE_SHEET_NUMBER)
sheet_ny.apply_for_all(cols_and_funcs, 2)

Index=2
Update price for RIO
Getting ticket object for RIO...
Index=2
Getting ticket object for RIO...
divs:
0.08257282
Index=2
Getting ticket object for RIO...
Index=2
Getting ticket object for RIO...
Index=2
Getting ticket object for RIO...
Index=2
Getting ticket object for RIO...
Index=3
Update price for HYMTF
Getting ticket object for HYMTF...
Index=3
Getting ticket object for HYMTF...
divs:
172.83951
Index=3
Getting ticket object for HYMTF...
Index=3
Getting ticket object for HYMTF...
Index=3
Getting ticket object for HYMTF...
Index=3
Getting ticket object for HYMTF...
Index=4
Update price for VZ
Getting ticket object for VZ...
Index=4
Getting ticket object for VZ...
divs:
0.0746123
Index=4
Getting ticket object for VZ...
Index=4
Getting ticket object for VZ...
Index=4
Getting ticket object for VZ...
Index=4
Getting ticket object for VZ...
Index=5
Update price for OKE
Getting ticket object for OKE...
Index=5
Getting ticket object for OKE...
divs:
0.065659195
Index=5
Getting ticket

Index=19
Getting ticket object for FDX...
Index=19
Getting ticket object for FDX...
Index=19
Getting ticket object for FDX...
Index=19
Getting ticket object for FDX...
Index=20
Update price for PG
Getting ticket object for PG...
Index=20
Getting ticket object for PG...
divs:
0.025125526
Index=20
Getting ticket object for PG...
Index=20
Getting ticket object for PG...
Index=20
Getting ticket object for PG...
Index=20
Getting ticket object for PG...
Index=21
Update price for JNJ
Getting ticket object for JNJ...
Index=21
Getting ticket object for JNJ...
divs:
0.029272715
Index=21
Getting ticket object for JNJ...
Index=21
Getting ticket object for JNJ...
Index=21
Getting ticket object for JNJ...
Index=21
Getting ticket object for JNJ...
Index=22
Update price for BAC
Getting ticket object for BAC...
Index=22
Getting ticket object for BAC...
divs:
0.03088392
Index=22
Getting ticket object for BAC...
Index=22
Getting ticket object for BAC...
Index=22
Getting ticket object for BAC...
Index=22


Index=30
Getting ticket object for BIDU...
Index=30
Getting ticket object for BIDU...
Index=30
Getting ticket object for BIDU...
Index=30
Getting ticket object for BIDU...
Index=31
Update price for JD
Getting ticket object for JD...
Index=31
Getting ticket object for JD...
divs:
0.0
dividendYield:
0.0187
Index=31
Getting ticket object for JD...
Index=31
Getting ticket object for JD...
Index=31
Getting ticket object for JD...
Index=31
Getting ticket object for JD...
Index=32
Update price for META
Getting ticket object for META...
Index=32
Getting ticket object for META...
divs:
0.0
dividendYield: key error
forwardAnnualDividendYield: key error
Index=32
Getting ticket object for META...
Index=32
Getting ticket object for META...
Index=32
Getting ticket object for META...
Index=32
Getting ticket object for META...
Index=33
Update price for PYPL
Getting ticket object for PYPL...
Index=33
Getting ticket object for PYPL...
divs:
0.0
dividendYield: key error
forwardAnnualDividendYield: key er

NoValue: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')